# OTHER SOLUTION TRIED DURING THE EXPERIMENTATION

## ZOO ATTACK USING ART

In [ ]:
import art.attacks.evasion.zoo as zoo

ZOO untargeted

In [ ]:
from art.estimators.classification import TensorFlowV2Classifier
import logging

#set log file for zoo attack
zoo.logging.basicConfig(filename='ZOO-Attack.log', level=logging.INFO)

loss_object = tf.keras.losses.sparse_categorical_crossentropy
optimizer = tf.keras.optimizers.Adam(0.001)
clf = TensorFlowV2Classifier(
    model=model,
    loss_object=loss_object,
    optimizer=optimizer,
    nb_classes=43,
    input_shape=(32, 32, 3),
    clip_values=(0, 1), #mx and min value allowe for features X. Since we initial rescaled the dataset, max=1 and min=0
)

In [ ]:
zooAtk=zoo.ZooAttack(classifier=clf, targeted=False, learning_rate=0.002, max_iter=1500, confidence=0.0,
                binary_search_steps=1, use_resize=False, initial_const=10, abort_early=True,
                use_importance=False, nb_parallel=128, batch_size=1, variable_h=0.0001)

#take images classified as stop sign with the lowest probability
lowest_prob_images_stop_sign_imgs=[i["image"] for i in lowest_prob_images_stop_sign][:100]

Perform the attack on the sample classified as stop sign with lowest prob in order to get an adversarial example

In [ ]:
x_test_adv = zooAtk.generate(np.array(lowest_prob_images_stop_sign_imgs))

Show adversarial images

In [ ]:
def show_images(imgs):
    # create figure 
    fig = plt.figure(figsize=(10, 7)) 

    # setting values to rows and column variables 
    rows = int(np.sqrt(imgs.shape[0]))+1
    columns = int(np.sqrt(imgs.shape[0]))+1
    
    for i in range(1,imgs.shape[0]+1):
        # Adds a subplot at the 1st position 
        fig.add_subplot(rows, columns, i) 

        # showing image 
        plt.imshow(imgs[i-1].reshape((32,32,3))) 
        plt.axis('off') 
        plt.title(i) 

In [ ]:
print("Generated Adversarial Examples")
show_images(x_test_adv)
plt.show()

Calculate succes rate and avg norm

In [ ]:
#calculate model's predictions for adversarial examples
pred_vector_adv=clf.predict(x_test_adv)
    
#get predicted label
pred_labels_adv=np.array([np.argmax(p) for p in pred_vector_adv])
    
#get number of adv samples not predicted as stop signs
success_advs=len([p for p in pred_labels_adv if p!=14])
    
#calculate success rate
success_rate=success_advs/pred_labels_adv.shape[0]
    
#calculate average norm
avg_norm=np.average(np.array([LA.norm(x_test_adv[i].ravel()-lowest_prob_images_stop_sign_imgs[i].ravel(),2) for i in range(x_test_adv.shape[0])]))
    
#return difference between success rate and avg norm 
#output is maximum when avg->0 and success_rate->1

print("Success rate=",success_rate)
print("avg_norm=",avg_norm)

### ZOO ART BAYES OPT

#### FIND BEST PARAMETERS FOR THE ZOO ATTACK WITH BAYESIAN OPTIMIZATION ON 20 EXAMPLES AND ON 500 ITERATIONS
In order to find best values for c, h and confidence parameter, such that the number of misclassified adversarial samples is highest, we can use an optimization method that finds the best values for previously parameters that maximize an objective function.

As objective function we create a function which has c,h and confidence as input paramaters and generates adversarial examples via zoo attack, use the model to classify advsersarial examples, getting the probability that the example is classified as a sto sign. Finally the function return 1 - the average probability of the vector of probabilities of all adversarial examples. This way the returned value will be low if most of adversarial examples are classified as stop signs and high otherwise.

As oprimization method we use the bayesian optimization method https://bayesian-optimization.github.io/BayesianOptimization/index.html which plans to find best values for paramaters in the lowest number of iterations possible

In [ ]:
from numpy import linalg as LA 

def zoo_performances(c,h,confidence):

    #print("initial const: ",c,", confidence: ", confidence,", h: ", h)
    
    #define the zoo attack object using input paramaters
    zooAtk=zoo.ZooAttack(classifier=clf, confidence=confidence, targeted=False, learning_rate=1e-2, max_iter=500,
                binary_search_steps=9, initial_const=c, abort_early=True, use_resize=False, 
                use_importance=False, nb_parallel=10, batch_size=1, variable_h=h) 

    #generate adversarial examples
    x_test_adv = zooAtk.generate(np.array(lowest_prob_images_stop_sign_imgs))
    
    #calculate model's predictions for adversarial examples
    pred_vector_adv=clf.predict(x_test_adv)
    
    #get predicted label
    pred_labels_adv=np.array([np.argmax(p) for p in pred_vector_adv])
    
    #get number of adv samples not predicted as stop signs
    success_advs=len([p for p in pred_labels_adv if p!=14])
    
    #calculate success rate
    success_rate=success_advs/pred_labels_adv.shape[0]
    
    #calculate average norm
    avg_norm=np.average(np.array([LA.norm(x_test_adv[i].ravel()-lowest_prob_images_stop_sign_imgs[i].ravel(),2) for i in range(x_test_adv.shape[0])]))
    
    #return difference between success rate and avg norm 
    #output is maximum when avg->0 and success_rate->1

    print("Success rate=",success_rate)
    print("avg_norm=",avg_norm)
    if avg_norm==0:
        return -100
    else:
        return success_rate-avg_norm

In [ ]:
from bayes_opt import BayesianOptimization, UtilityFunction

# define thebounded space for parameters c,h,confidence
#pbounds = {'c': (0,1), 'h': (0,1), 'confidence':(-5,5)}
#decided to try only positive confidence since with negative values, generated adversarial examples has a zoo_performancs of -100
#pbounds = {'c': (0.001,0.99), 'h': (0.001,0.99), 'confidence':(0,5)}
#decide to lock c=(0.001,0.1) since best results (sucecss_rate>60%) has reached with this value for c
#pbounds = {'c': (0.001,0.1), 'h': (0.001,0.99), 'confidence':(0,5)}
#decide to try only c=0.001
pbounds = {'c': (0.001,0.001), 'h': (0.001,0.99), 'confidence':(0,5)}

#define the optimizer
optimizer = BayesianOptimization(
    f=zoo_performances,
    pbounds=pbounds,
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
)

Register some past values found by running bayes_opt before

In [ ]:
past_bayes_opt_results=[
    {'params':{'c':0.29,'h':0.25,'confidence':1.97},'target':-1.921},
    {'params':{'c':0.15,'h':0.22,'confidence':3.63},'target':-1.883},
    {'params':{'c':1,'h':1,'confidence':5},'target':-2.02},
    {'params':{'c':1,'h':1,'confidence':2.85},'target':-1.906},
    {'params':{'c':0.32,'h':0.33,'confidence':-1.03},'target':-100},
    {'params':{'c':0.55,'h':0.07,'confidence':4.2},'target':-1.945},
    {'params':{'c':1,'h':1,'confidence':-5},'target':-100},
    {'params':{'c':0.93,'h':0.02,'confidence':2.63},'target':-2.089},
    {'params':{'c':0,'h':1,'confidence':5},'target':-100},
    {'params':{'c':0.96,'h':0.91,'confidence':3.82},'target':-1.837},
    {'params':{'c':0.97,'h':1,'confidence':1.22},'target':-1.898},
    {'params':{'c':0.413,'h':0.95,'confidence':-376},'target':-100},
    {'params':{'c':0.62,'h':0.2,'confidence':1.67},'target':-1.975},
    {'params':{'c':0.5,'h':0.98,'confidence':2.43},'target':-2.091},
    {'params':{'c':0.87,'h':0.06,'confidence':-4.2},'target':-100},
    {'params':{'c':0.99,'h':0.05,'confidence':4.93},'target':-1.987},
    {'params':{'c':0,'h':1,'confidence':1.08},'target':-100},
    {'params':{'c':0.97,'h':0.76,'confidence':1.45},'target':-1.876},  #success_rate 60%
    {'params':{'c':0.89,'h':0.54,'confidence':1.53},'target':-1.868},
    {'params':{'c':0.45,'h':0.97,'confidence':-2.73},'target':-100},
    {'params':{'c':0.25,'h':0.51,'confidence':3.57},'target':-1.726},  #success rate 60%
    {'params':{'c':0.46,'h':0.64,'confidence':0.38},'target':-1.856},
    {'params':{'c':0.99,'h':0.001,'confidence':0.48},'target':-2.138},
    {'params':{'c':0.27,'h':0.74,'confidence':3.35},'target':-1.807},
    {'params':{'c':0.33,'h':0.84,'confidence':0.36},'target':-1.842},  #success rate 60%
    {'params':{'c':0.64,'h':0.21,'confidence':-3.48},'target':-100},
    {'params':{'c':0.93,'h':0.03,'confidence':-3.58},'target':-100},
    {'params':{'c':0.4,'h':0.88,'confidence':3.5},'target':-1.897},
    {'params':{'c':0.95,'h':0.95,'confidence':0.14},'target':-1.928},   #success rate 55%
    {'params':{'c':0.99,'h':0.001,'confidence':3.65},'target':-2.023},  #success rate 55%
    {'params':{'c':0.001,'h':0.001,'confidence':2.698},'target':-1.654},   #succes rate 55%
    {'params':{'c':0.99,'h':0.54,'confidence':2.12},'target':-1.858},  #sucess rate 55%
    {'params':{'c':0.99,'h':0.001,'confidence':1.18},'target':-2.193},
    {'params':{'c':0.99,'h':0.5,'confidence':4.44},'target':-1.896},  #success rate 55%
    {'params':{'c':0.99,'h':0.65,'confidence':0.71},'target':-1.996},
    {'params':{'c':0.001,'h':0.99,'confidence':-0.19},'target':-1.473},  #succes rate 40%
    {'params':{'c':0.08,'h':0.002,'confidence':0.32},'target':-1.887},  #sucess rate 55%
    {'params':{'c':0.54,'h':0.40,'confidence':2.81},'target':-1.991},  #success rate 50%
    {'params':{'c':0.68,'h':0.9,'confidence':-4.61},'target':-100},
    {'params':{'c':0.83,'h':0.13,'confidence':-3.31},'target':-100},
    {'params':{'c':0.46,'h':0.82,'confidence':3.02},'target':-1.888},  #success rate 50%
    {'params':{'c':0.04,'h':0.42,'confidence':3.941},'target':-1.629}, #success rate 55%
    {'params':{'c':0.55,'h':0.92,'confidence':-3.22},'target':-100},
    {'params':{'c':0.51,'h':0.001,'confidence':2.23},'target':-2.051},  #success rate 60%
    {'params':{'c':0.001,'h':0.745,'confidence':2.678},'target':-1.216},  #success rate 65%
    {'params':{'c':0.44,'h':0.99,'confidence':-0.04},'target':-1.94},  #success rate 50%
    {'params':{'c':0.43,'h':0.001,'confidence':0.84},'target':-2.112},  #success rate 50%
    {'params':{'c':0.001,'h':0.001,'confidence':1.842},'target':-1.683},   #success rate 50%
    {'params':{'c':0.001,'h':0.001,'confidence':4.306},'target':-1.614},  #success rate 65%
    {'params':{'c':0.001,'h':0.57,'confidence':0.049},'target':-1.319},  #success rate 50%
    {'params':{'c':0.99,'h':0.99,'confidence':2.03},'target':-1.982},  #success rate 50%
    {'params':{'c':0.99,'h':0.001,'confidence':4.34},'target':-1.981},  #success rate 55%
    {'params':{'c':0.98,'h':0.5,'confidence':3.35},'target':-1.991},  #success rate 50%
    {'params':{'c':0.32,'h':0.56,'confidence':4.5},'target':-1.906},  #success rate 45%
    {'params':{'c':0.13,'h':0.79,'confidence':2.41},'target':-1.813},  #success rate 50%
    {'params':{'c':0.71,'h':0.77,'confidence':2.47},'target':-2.004},  #success rate 50%
    {'params':{'c':0.64,'h':0.87,'confidence':-3.67},'target':-100},
    {'params':{'c':0.77,'h':0.73,'confidence':0.93},'target':-1.878},  #success rate 55%
    {'params':{'c':0.36,'h':0.001,'confidence':5.0},'target':-1.906},  #success rate 55%
    {'params':{'c':0.99,'h':0.99,'confidence':4.45},'target':-2.011},  #success rate 45%
    {'params':{'c':0.57,'h':0.19,'confidence':4.68},'target':-1.993},  #success rate 45%
    {'params':{'c':0.83,'h':0.99,'confidence':0.52},'target':-1.884},  #success rate 55%
    {'params':{'c':0.52,'h':0.65,'confidence':4.11},'target':-1.97},  #success rate 50%
    {'params':{'c':0.99,'h':0.55,'confidence':5.0},'target':-1.922},  #success rate 55%
    {'params':{'c':0.99,'h':0.001,'confidence':3.12},'target':-2.046},  #success rate 55%
    {'params':{'c':0.53,'h':0.001,'confidence':0.19},'target':-2.129},  #success rate 45%
    {'params':{'c':0.98,'h':0.02,'confidence':1.73},'target':-2.135},  #success rate 40%
    {'params':{'c':0.27,'h':0.31,'confidence':-1.48},'target':-100},
    {'params':{'c':0.31,'h':0.98,'confidence':1.74},'target':-1.87},  #success rate 55%
    {'params':{'c':0.91,'h':0.92,'confidence':1.96},'target':-1.93},  #success rate 50%
    {'params':{'c':0.08,'h':0.38,'confidence':-3.76},'target':-100},
    {'params':{'c':0.96,'h':0.17,'confidence':-0.35},'target':-2.06},  #success rate 45%
    {'params':{'c':0.99,'h':0.34,'confidence':0.02},'target':-1.943},  #success rate 50%
    {'params':{'c':0.99,'h':0.99,'confidence':-0.56},'target':-2.023},  #success rate 50%
    {'params':{'c':0.41,'h':0.001,'confidence':3.71},'target':-2.111},  #success rate 40%
    {'params':{'c':0.99,'h':0.001,'confidence':-0.08},'target':-2.11},  #success rate 55%
    {'params':{'c':0.001,'h':0.99,'confidence':3.236},'target':-1.284},  #success rate 60%
    {'params':{'c':0.001,'h':0.30,'confidence':3.14},'target':-1.329},  #success rate 55%
    {'params':{'c':0.45,'h':0.94,'confidence':2.0},'target':-1.867},  #success rate 55%
    {'params':{'c':0.99,'h':0.56,'confidence':-0.33},'target':-2.004},  #success rate 45%
    {'params':{'c':0.001,'h':0.001,'confidence':4.8},'target':-1.51},  #success rate 60%
    {'params':{'c':0.97,'h':0.989,'confidence':3.32},'target':-2.008},  #success rate 45%
    {'params':{'c':0.006,'h':0.93,'confidence':-3.6},'target':-100},
    {'params':{'c':0.45,'h':0.2,'confidence':-2.46},'target':-100},
    {'params':{'c':0.27,'h':0.299,'confidence':-3.66},'target':-100},
    {'params':{'c':0.77,'h':0.5,'confidence':-2.82},'target':-100},
    {'params':{'c':0.69,'h':0.93,'confidence':-3.515},'target':-100},
    {'params':{'c':0.42,'h':0.001,'confidence':3.109},'target':-2.024},  #success rate 55%
    {'params':{'c':0.98,'h':0.988,'confidence':-1.213},'target':-100},
    {'params':{'c':0.001,'h':0.99,'confidence':3.832},'target':-1.236},  #success rate 60%
    {'params':{'c':0.98,'h':0.956,'confidence':-0.28},'target':-2.017},  #success rate 40%
    {'params':{'c':0.002127,'h':0.3754,'confidence':2.327},'target':-1.18},  #success rate 70%
    {'params':{'c':0.02,'h':0.97,'confidence':2.71},'target':-1.502},  #success rate 50%
    {'params':{'c':0.98,'h':0.53,'confidence':2.78},'target':-1.977},  #success rate 45%
    {'params':{'c':0.687,'h':0.99,'confidence':1.59},'target':-2.037},  #success rate 35%
    {'params':{'c':0.35,'h':0.003,'confidence':4.625},'target':-2.028},  #success rate 50%
    {'params':{'c':0.217,'h':0.1,'confidence':2.638},'target':-1.94},  #success rate 40%
    {'params':{'c':0.53,'h':0.49,'confidence':2.081},'target':-1.843},  #success rate 55%
    {'params':{'c':0.8,'h':0.78,'confidence':3.281},'target':-1.972},  #success rate 50%
    {'params':{'c':0.866,'h':0.77,'confidence':1.932},'target':-1.893},  #success rate 50%
    {'params':{'c':0.883,'h':0.25,'confidence':3.748},'target':-1.933},  #success rate 50%
    {'params':{'c':0.001,'h':0.99,'confidence':2.112},'target':-1.216},  #success rate 60%
    {'params':{'c':0.001,'h':0.001,'confidence':0.0},'target':-1.304},  #success rate 60%
    {'params':{'c':0.36,'h':0.001,'confidence':1.374},'target':-1.928},  #success rate 45%
    {'params':{'c':0.001,'h':0.001,'confidence':3.396},'target':-1.516},  #success rate 50%
    {'params':{'c':0.788,'h':0.26,'confidence':0.9292},'target':-2.014},  #success rate 50%
    {'params':{'c':0.001,'h':0.99,'confidence':0.066},'target':-1.437},  #success rate 45%
    {'params':{'c':0.99,'h':0.001,'confidence':2.187},'target':-1.876},  #success rate 55%
    {'params':{'c':0.21,'h':0.32,'confidence':4.317},'target':-2.021},  #success rate 40%
    {'params':{'c':0.01,'h':0.008,'confidence':2.264},'target':-1.664},  #success rate 45%
    {'params':{'c':0.001,'h':0.72,'confidence':3.596},'target':-1.366},  #success rate 50%
    {'params':{'c':0.08,'h':0.22,'confidence':3.75},'target':-1.591},  #success rate 60%
    {'params':{'c':0.078,'h':0.427,'confidence':0.409},'target':-1.814},  #success rate 40%
    {'params':{'c':0.02,'h':0.18,'confidence':0.615},'target':-1.592},  #success rate 50%
    {'params':{'c':0.08,'h':0.266,'confidence':2.144},'target':-1.762},  #success rate 45%
    {'params':{'c':0.05,'h':0.865,'confidence':2.28},'target':-1.679},  #success rate 50%
    {'params':{'c':0.001,'h':0.001,'confidence':1.104},'target':-1.399},  #success rate 55%
    {'params':{'c':0.1,'h':0.64,'confidence':1.889},'target':-1.769},  #success rate 50%
    {'params':{'c':0.1,'h':0.26,'confidence':0.134},'target':-1.75},  #success rate 50%
    {'params':{'c':0.001,'h':0.001,'confidence':3.896},'target':-1.363},  #success rate 55%
    {'params':{'c':0.1,'h':0.82,'confidence':0.03},'target':-1.71},  #success rate 60%
    {'params':{'c':0.001,'h':0.001,'confidence':1.475},'target':-1.372},  #success rate 60%
    {'params':{'c':0.1,'h':0.77,'confidence':4.074},'target':-1.718},  #success rate 50%
    {'params':{'c':0.02,'h':0.005,'confidence':0.75},'target':-1.586},  #success rate 55%
    {'params':{'c':0.001,'h':0.34,'confidence':2.746},'target':-1.388},  #success rate 60%
    {'params':{'c':0.001,'h':0.68,'confidence':3.058},'target':-1.207},  #success rate 60%
    {'params':{'c':0.001,'h':0.25,'confidence':4.397},'target':-1.448},  #success rate 45%
    {'params':{'c':0.001,'h':0.88,'confidence':3.144},'target':-1.321},  #success rate 60%
    {'params':{'c':0.001,'h':0.2452,'confidence':1.399},'target':-1.17},  #success rate 70%
    {'params':{'c':0.001,'h':0.624,'confidence':0.159},'target':-1.219},  #success rate 60%
    {'params':{'c':0.001,'h':0.39,'confidence':1.624},'target':-1.349},  #success rate 50%
    {'params':{'c':0.001,'h':0.22,'confidence':1.742},'target':-1.305},  #success rate 60%
    {'params':{'c':0.001,'h':0.001,'confidence':3.042},'target':-1.387},  #success rate 55%
    {'params':{'c':0.001,'h':0.09,'confidence':1.393},'target':-1.36},  #success rate 55%
    {'params':{'c':0.001,'h':0.835,'confidence':4.048},'target':-1.287},  #success rate 55%
    {'params':{'c':0.001,'h':0.418,'confidence':2.37},'target':-1.372},  #success rate 55%
    {'params':{'c':0.001,'h':0.259,'confidence':4.419},'target':-1.268},  #success rate 65%
    {'params':{'c':0.001,'h':0.7674,'confidence':1.753},'target':-1.247},  #success rate 65%
    {'params':{'c':0.001,'h':0.245,'confidence':0.281},'target':-1.459},  #success rate 45%
    {'params':{'c':0.001,'h':0.55,'confidence':4.26},'target':-1.339},  #success rate 60%
    {'params':{'c':0.001,'h':0.4,'confidence':3.483},'target':-1.43},  #success rate 45%
    {'params':{'c':0.001,'h':0.004,'confidence':4.564},'target':-1.386},  #success rate 60%
    {'params':{'c':0.001,'h':0.834,'confidence':1.954},'target':-1.292},  #success rate 60%
    {'params':{'c':0.001,'h':0.712,'confidence':3.882},'target':-1.295},  #success rate 55%
    {'params':{'c':0.001,'h':0.99,'confidence':2.447},'target':-1.22},  #success rate 65%
    {'params':{'c':0.001,'h':0.001,'confidence':5.0},'target':-1.362},  #success rate 60%
    {'params':{'c':0.001,'h':0.989,'confidence':3.556},'target':-1.39},  #success rate 50%
    {'params':{'c':0.001,'h':0.18,'confidence':2.477},'target':-1.394},  #success rate 55%
    {'params':{'c':0.001,'h':0.881,'confidence':0.706},'target':-1.209},  #success rate 60%
    {'params':{'c':0.001,'h':0.76,'confidence':0.345},'target':-1.309},  #success rate 55%
    {'params':{'c':0.001,'h':0.324,'confidence':2.487},'target':-1.17},  #success rate 65%
    {'params':{'c':0.001,'h':0.779,'confidence':2.322},'target':-1.322},  #success rate 60%
    {'params':{'c':0.001,'h':0.44,'confidence':1.301},'target':-1.255},  #success rate 60%
    {'params':{'c':0.001,'h':0.6664,'confidence':0.659},'target':-1.36},  #success rate 60%
    {'params':{'c':0.001,'h':0.92,'confidence':0.5345},'target':-1.191},  #success rate 60%
    {'params':{'c':0.001,'h':0.088,'confidence':2.345},'target':-1.446},  #success rate 45%
    {'params':{'c':0.001,'h':0.2,'confidence':2.071},'target':-1.307},  #success rate 60%
    {'params':{'c':0.001,'h':0.71,'confidence':0.8464},'target':-1.238},  #success rate 65%
    {'params':{'c':0.001,'h':0.21,'confidence':3.622},'target':-1.325},  #success rate 55%
    {'params':{'c':0.001,'h':0.22,'confidence':3.691},'target':-1.339},  #success rate 60%
    {'params':{'c':0.001,'h':0.375,'confidence':1.013},'target':-1.278},  #success rate 65%
    {'params':{'c':0.001,'h':0.799,'confidence':0.6734},'target':-1.309},  #success rate 55%
    {'params':{'c':0.001,'h':0.554,'confidence':0.8276},'target':-1.238},  #success rate 60%
    {'params':{'c':0.001,'h':0.322,'confidence':1.218},'target':-1.445},  #success rate 50%
    {'params':{'c':0.001,'h':0.687,'confidence':0.779},'target':-1.362},  #success rate 50%
    {'params':{'c':0.001,'h':0.7,'confidence':1.79},'target':-1.289},  #success rate 65%
    {'params':{'c':0.001,'h':0.563,'confidence':0.499},'target':-1.475},  #success rate 45%
    {'params':{'c':0.001,'h':0.7306,'confidence':2.01},'target':-1.194},  #success rate 60%
    {'params':{'c':0.001,'h':0.212,'confidence':4.807},'target':-1.377},  #success rate 50%
    {'params':{'c':0.001,'h':0.77,'confidence':4.072},'target':-1.28},  #success rate 55%
    {'params':{'c':0.001,'h':0.46,'confidence':4.036},'target':-1.298},  #success rate 55%
    {'params':{'c':0.001,'h':0.68,'confidence':4.169},'target':-1.246},  #success rate 60%
    {'params':{'c':0.001,'h':0.557,'confidence':0.93},'target':-1.446},  #success rate 45%
    {'params':{'c':0.001,'h':0.17,'confidence':4.647},'target':-1.307},  #success rate 60%
    {'params':{'c':0.001,'h':0.39,'confidence':1.44},'target':-1.447},  #success rate 50%
    {'params':{'c':0.001,'h':0.886,'confidence':0.62},'target':-1.371},  #success rate 55%
    {'params':{'c':0.001,'h':0.8743,'confidence':0.07},'target':-1.33},  #success rate 60%
    {'params':{'c':0.001,'h':0.66,'confidence':2.044},'target':-1.251},  #success rate 60%
    {'params':{'c':0.001,'h':0.547,'confidence':3.286},'target':-1.299},  #success rate 65%
    {'params':{'c':0.001,'h':0.6,'confidence':2.611},'target':-1.441},  #success rate 50%
    {'params':{'c':0.001,'h':0.43,'confidence':2.575},'target':-1.213},  #success rate 60%
    {'params':{'c':0.001,'h':0.4,'confidence':5.0},'target':-1.446},  #success rate 45%
    {'params':{'c':0.001,'h':0.99,'confidence':4.442},'target':-1.264},  #success rate 55%
    {'params':{'c':0.001,'h':0.633,'confidence':4.662},'target':-1.389},  #success rate 45%
    {'params':{'c':0.001,'h':0.33,'confidence':0.9622},'target':-1.353},  #success rate 60%
    {'params':{'c':0.001,'h':0.27,'confidence':4.785},'target':-1.309},  #success rate 60%
    {'params':{'c':0.001,'h':0.766,'confidence':1.63},'target':-1.344},  #success rate 55%
    {'params':{'c':0.001,'h':0.6,'confidence':1.057},'target':-1.304},  #success rate 60%
    {'params':{'c':0.001,'h':0.11,'confidence':1.098},'target':-1.25},  #success rate 65%
    {'params':{'c':0.001,'h':0.433,'confidence':2.573},'target':-1.26},  #success rate 60%
    {'params':{'c':0.001,'h':0.524,'confidence':1.959},'target':-1.433},  #success rate 45%
    {'params':{'c':0.001,'h':0.864,'confidence':4.341},'target':-1.345},  #success rate 55%
    {'params':{'c':0.001,'h':0.52,'confidence':4.565},'target':-1.386},  #success rate 50%
    {'params':{'c':0.001,'h':0.617,'confidence':1.62},'target':-1.298},  #success rate 60%
]

for p in past_bayes_opt_results:
    optimizer.register(params=p['params'], target=p['target'])

In [ ]:
#perform the optimiztion
#After getting lots of points, we use UCB with low value for kappa parameter (0.1 instead of 2.576 which was the default value for kappa) 
#near the actutal found maximum of the function in order to prefer exploitation and focusing on points 


acquisition_function = UtilityFunction(kind="ucb", kappa=0.1)
optimizer.maximize(
    init_points=5,  #step of random exploration
    n_iter=10, #n_iter that the optimization algorithm has to perform
    acquisition_function=acquisition_function
)

#### BAYESIAN OPTIMIZATION USING 1000 ITERATIONS AND 100 EXAMPLES

In [ ]:
from numpy import linalg as LA 

def zoo_performances_2(c,h,confidence,lr):

    #print("initial const: ",c,", confidence: ", confidence,", h: ", h)
    
    #define the zoo attack object using input paramaters
    zooAtk=zoo.ZooAttack(classifier=clf, confidence=confidence, targeted=False, learning_rate=lr, max_iter=1000,
                binary_search_steps=9, initial_const=c, abort_early=True, use_resize=False, 
                use_importance=False, nb_parallel=10, batch_size=1, variable_h=h) 

    #generate adversarial examples
    x_test_adv = zooAtk.generate(np.array(lowest_prob_images_stop_sign_imgs))
    
    #calculate model's predictions for adversarial examples
    pred_vector_adv=clf.predict(x_test_adv)
    
    #get predicted label
    pred_labels_adv=np.array([np.argmax(p) for p in pred_vector_adv])
    
    #get number of adv samples not predicted as stop signs
    success_advs=len([p for p in pred_labels_adv if p!=14])
    
    #calculate success rate
    success_rate=success_advs/pred_labels_adv.shape[0]
    
    #calculate average norm
    avg_norm=np.average(np.array([LA.norm(x_test_adv[i].ravel()-lowest_prob_images_stop_sign_imgs[i].ravel(),2) for i in range(x_test_adv.shape[0])]))
    
    #return difference between success rate and avg norm 
    #output is maximum when avg->0 and success_rate->1

    
    #generate and save image of adv examples
    pred_imgs_adv=[]


    for i in range(pred_vector_adv.shape[0]):
        #calculate predicted label
        label_pred_adv = np.argmax(pred_vector_adv[i])
        
        #calulate predicted label proba
        prob_pred_adv = np.max(pred_vector_adv[i])
        
        #calculate stop sign proba
        stop_sign_proba = pred_vector_adv[i][14]
        
        #calculate norm l2 of the difference between adversarial example and original example
        diff=LA.norm(x_test_adv[i].ravel()-lowest_prob_images_stop_sign_imgs[i].ravel(),2)
        
        pred_imgs_adv.append({"image":x_test_adv[i],"predicted_label":label_pred_adv,"real_label":14,
                              "predicted_proba":prob_pred_adv,"stop_sign_proba":stop_sign_proba,"difference":diff})
    
    show_images_adv_pred(np.array(pred_imgs_adv),18,15,True,"./bayes_opt_res/adv_img_res/attack_c_"+str(c)+"_conf_"+str(confidence)+"_h_"+str(h)+"_lr_"+str(lr))

    if avg_norm==0:
        result = -100
    else:
        result = success_rate-avg_norm
    
    print("Success rate=",success_rate)
    print("avg_norm=",avg_norm)
    print("c=",c,"confidence=",confidence,"h=",h,"lr=",lr,"target=",result)
    
    return result

In [ ]:
from bayes_opt import BayesianOptimization, UtilityFunction
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs

pbounds = {'c': (0,1), 'h': (0.001,0.99), 'confidence':(0,10), 'lr':(0,0.1)}

#define the optimizer
optimizer_2 = BayesianOptimization(
    f=zoo_performances_2,
    pbounds=pbounds,
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
)

load_logs(optimizer_2, logs=["./bayes_opt_res/bayes_opt_1000_iter_100_ex.log.json"]);

logger = JSONLogger(path="./bayes_opt_res/bayes_opt_1000_iter_100_ex.log",reset=False)
optimizer_2.subscribe(Events.OPTIMIZATION_STEP, logger)

In [ ]:
#perform the optimiztion   
#we use 5 as kappa in order to prefer nor exploitation and exploration

acquisition_function = UtilityFunction(kind="ucb", kappa=5)
optimizer_2.maximize(
    init_points=0,  #step of random exploration to do before starting the effectively search of the maximum of the funcion
    #initial points was set to 5
    n_iter=100, #n_iter that the optimization algorithm has to perform after the initial random exploration steps
    acquisition_function=acquisition_function
)

## ZOO WITH AUTOZOOM LIBRARY

## ZOO ATTACK BAYESIAN OPTIMIZATION WITHOUT KNOW PARAMETERS SPACE

In [ ]:
from numpy import linalg as LA 

class ZOO_performance_func:
    def __init__(self, bounds=None, sd=None):
        self.input_dim = 4
        self.bounds = [(1e-4,1),(0,1),(1e-4,1e-2)]   #bonds for initial_const,confidence and learning rate

    def func(self,x):
        c,confidence,lr = x
        
        #list which contains all adv examples
        advs=[]

        with tf.Session() as sess:
            #initialize variables (prevent FailedPreconditionError error)
            init_op = tf.initialize_all_variables()
            sess.run(init_op)

            model.load_weights("Model.weights_zoo.h5")

            mod=Modello(model)

            args={"batch_size":128,"img_resize":32,"max_iterations":1500,"print_every":1000,"switch_iterations":100,
                 "init_const":c,"attack_type":"untargeted","use_tanh":False,"lr":lr,"confidence":confidence}

            blackbox_attack = ZOO(sess,mod,args)

            labelss=np.zeros(43)
            labelss[14]=1  #one-hot encoded label


            #cicle through all examples
            for i in lowest_prob_images_stop_sign_imgs:
                adv = blackbox_attack.attack(i,labelss)

                advs.append(adv)


            advs=np.array(advs) 

        #generate adversarial examples
        x_test_adv = advs

        #calculate model's predictions for adversarial examples
        pred_vector_adv=mod.model.predict(x_test_adv)

        #get predicted label
        pred_labels_adv=np.array([np.argmax(p) for p in pred_vector_adv])

        #get number of adv samples not predicted as stop signs
        success_advs=len([p for p in pred_labels_adv if p!=14])

        #calculate success rate
        success_rate=success_advs/pred_labels_adv.shape[0]

        #calculate average norm
        avg_norm=np.average(np.array([LA.norm(x_test_adv[i].ravel()-lowest_prob_images_stop_sign_imgs[i].ravel(),2) for i in range(x_test_adv.shape[0])]))

        #return difference between success rate and avg norm 
        #output is maximum when avg->0 and success_rate->1


        #generate and save image of adv examples
        pred_imgs_adv=[]


        for i in range(pred_vector_adv.shape[0]):
            #calculate predicted label
            label_pred_adv = np.argmax(pred_vector_adv[i])

            #calulate predicted label proba
            prob_pred_adv = np.max(pred_vector_adv[i])

            #calculate stop sign proba
            stop_sign_proba = pred_vector_adv[i][14]

            #calculate norm l2 of the difference between adversarial example and original example
            diff=LA.norm(x_test_adv[i].ravel()-lowest_prob_images_stop_sign_imgs[i].ravel(),2)

            pred_imgs_adv.append({"image":x_test_adv[i],"predicted_label":label_pred_adv,"real_label":14,
                                  "predicted_proba":prob_pred_adv,"stop_sign_proba":stop_sign_proba,"difference":diff})

        show_images_adv_pred(np.array(pred_imgs_adv),20,18,True,"./bayes_opt_res/adv_img_res/attack_c_"+str(c)+"_conf_"+str(confidence)+"_lr_"+str(lr))

        if avg_norm==0:
            result = -100
        else:
            result = success_rate-avg_norm

        print("\n\nSuccess rate=",success_rate)
        print("avg_norm=",avg_norm)
        print("c=",c,"confidence=",confidence,"lr=",lr,"target=",result,"\n\n")

        return result
        

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

import numpy as np
from numpy.linalg import pinv
from scipy.optimize import minimize

import time

In [ ]:
# Declare the model one wants to investigate
bm_function = 'Hartman3'


myfunction = ZOO_performance_func()

func = myfunction.func
bounds_all = np.asarray(myfunction.bounds)
box_len = 0.2*np.max(bounds_all[:, 1] - bounds_all[:, 0]) # Box length to be 20% of the original box
print('Box length: {}'.format(box_len))


# Experiments with the proposed alogrithm
epsilon = 0.05 # Set the epsilon-accuracy condition
iter_mul = 10 # Set the evaluation budget, i.e. iter_mul x dim
max_exp = 30 # Set the number of experiments
Y_max_all = []
bounds_final_all = []
X_init_all = []
Y_init_all = []
gp_all = []
regret_all = []
r_optimal_all = []
time_all = []
for n_exp in range(max_exp):

    print('Test Function: ' + bm_function)
    print('Experiment {}:'.format(n_exp))

    # Set seed and initial points
    np.random.seed(n_exp)
    n_init_points = 3*myfunction.input_dim
    gp_exp = []

    # Randomly generate search space
    b_init_center = np.zeros((len(bounds_all), 1))
    for i in range(len(bounds_all)):
        b_init_center[i] = np.random.uniform(bounds_all[i][0]+box_len/2,
                                             bounds_all[i][1]-box_len/2)

    b_init_lower = b_init_center - box_len/2
    b_init_upper = b_init_center + box_len/2
    bounds_user = np.asarray([b_init_lower.ravel(), b_init_upper.ravel()]).T
    bounds = bounds_user.copy()
    b_init_center = np.mean(bounds, axis=1)
    print('Bounds: {}'.format(bounds))

    # Generate and normalize input, ouput
    temp = [np.random.uniform(x[0], x[1], size=n_init_points) for x in bounds]
    temp = np.asarray(temp)
    temp = temp.T
    X_init = list(temp.reshape((n_init_points, -1)))
    print("\n\nParams:",X_init[0])
    Y_init = func(X_init[0])
    for i in range(1, len(X_init)):
        print("\n\nParams:",X_init[i])
        temp = func(X_init[i])
        Y_init = np.append(Y_init, temp)
    Y = (Y_init-np.mean(Y_init))/(np.max(Y_init)-np.min(Y_init))

    # Start measure time
    start = time.time()

    # Fit Gaussian Process
    # Be careful with the fitting methodology (it can be stuck in local minima)
    # Multi-start?
    ls_prior = [0.01, 0.1, 1, 10]
    ml_value = np.zeros(len(ls_prior))
    gp_temp = []
    for idx, ls in enumerate(ls_prior):
        kernel = 1.0 * RBF(length_scale=ls)
        gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=25)
        gp.fit(X_init, Y)
        gp_temp.append(gp)
        ml_value[idx] = gp.log_marginal_likelihood(gp.kernel_.theta)
    gp = gp_temp[np.min(np.argmax(ml_value))]
    gp_exp.append(gp)
    print('Kernel {} with Log Marginal Likelihood {}'.format(gp.kernel_, gp.log_marginal_likelihood(gp.kernel_.theta)))

    # Extract Gaussian Process hyper-parameters
    # kernel_k2 is the length-scale
    # theta_n is the scale factor
    kernel_k1 = gp.kernel_.get_params()['k1__constant_value']
    kernel_k2 = gp.kernel_.get_params()['k2__length_scale']
    theta_n = np.sqrt(kernel_k1)

    # Re-arrange the initial random observations
    max_iter = iter_mul*myfunction.input_dim
    Y_max = np.zeros((max_iter+n_init_points, 1))
    for i in range(n_init_points):
        Y_max[i] = np.max(Y_init[0:i+1])

    # Set some parameters for b_n and the evaluation budget
    max_iter_i = iter(range(1, max_iter+1))
    nu = 0.2 
    sigma = 0.1
    
    # List to store important values
    regret_iter = []
    r_optimal_iter = []
    bounds_iter = []
    iter_adjust = 0
    regret = 10*epsilon
    for n_iter_i in max_iter_i:

        n_iter = n_iter_i - iter_adjust
        print("Iteration {}".format(n_iter))

        # Compute the acquisition function at the observation points
        # b_n is found by using the correct formula
        lengthscale = gp.kernel_.get_params()['k2__length_scale']
        thetan_2 = gp.kernel_.get_params()['k1__constant_value']
        radius = np.abs(np.max(bounds[:, 1]-bounds[:, 0]))
        b = 1/2*np.sqrt(2)*np.sqrt(thetan_2)/lengthscale
        a = 1
        tau_n = (4*myfunction.input_dim+4)*np.log(n_iter) + 2*np.log(2*np.pi**2/(3*sigma)) \
                + 2*myfunction.input_dim*np.log(myfunction.input_dim*b*radius*np.sqrt(np.log(4*myfunction.input_dim*a/sigma)))
        b_n = np.sqrt(np.abs(nu*tau_n))
        y_init_mean, y_init_std = gp.predict(X_init, return_std=True)
        acq_init = y_init_mean.ravel() + b_n*y_init_std.ravel()
        print('b_n is: {}'.format(b_n))

        # Optimize the acquisition function
        x_max = acq_maximize_fixopt(gp, b_n, bounds)
        max_acq = acq_gp(x_max, gp, b_n)
        y_acq = acq_gp(np.asarray(X_init), gp, b_n)

        # Compute the maximum regret
        X_init_temp = X_init.copy()
        X_init_temp.append(x_max)
        Y_lcb_temp = acq_lcb(np.asarray(X_init_temp), gp, b_n)
        regret = max_acq - np.max(Y_lcb_temp)
        regret_iter.append(regret)
        print('Regret: {}'.format(regret))
        
        # Check if regret < 0, redo the optimization, typically redo the optimization
        # with starting point in X_init_temp
        indices_max_X = np.argsort(Y_lcb_temp)[::-1]
        n_search_X = np.min([20, len(indices_max_X)])
        if (regret + 1/n_iter**2 < 0):
            print('Regret < 0, redo the optimization')
            for i in range(n_search_X):
                x_try = X_init_temp[i]
                # Running L-BFGS-B from x_max
                res = minimize(lambda x: -acq_gp(x, gp, b_n),
                               x_try, bounds=bounds, method="L-BFGS-B")
            
                # Store it if better than previous minimum(maximum).
                if max_acq is None or -res.fun[0] >= max_acq:
                    x_max = res.x
                    max_acq = -res.fun[0]
                
                # Recompute regret
                regret = max_acq - np.max(Y_lcb_temp)
                print('Regret: {}'.format(regret))
            

        # Expand if regret < epsilon or the first iteration
        if (regret <= epsilon - 1/n_iter**2) | (n_iter_i == 1):
            print('Expanding bounds')

            # Expand the search space based on analytical formula in Theorem 1
            K = gram_matrix(X_init, 1, kernel_k2)
            K_inv = pinv(K)
            b = np.matmul(K_inv, Y)
            b_pos = b[b >= 0]
            b_neg = b[b <= 0]
            U, Sigma, V = np.linalg.svd(K_inv)
            lambda_max = np.max(Sigma)
            n = len(X_init)
            gamma = min(0.25*epsilon/max(np.sum(b_pos), -np.sum(b_neg)),
                        1/b_n*np.sqrt((0.5*epsilon*b_n*theta_n-0.0625*epsilon**2)/(n*lambda_max)))

            if (gamma > 1):
                raise ValueError('Gamma needs to be smaller than 1. Something wrong!')

            # Find d_{gamma}
            # For SE kernel, it's easy to find values of r to make k(r) < gamma
            scale_l = np.sqrt(-2*np.log(gamma/kernel_k1))
            print('The scale is: {}'.format(scale_l))

            bounds_ori_all = []
            for n_obs in range(len(acq_init)):
                X0 = X_init[n_obs]
                # Set the rectangle bounds
                bounds_ori_temp = np.asarray((X0 - scale_l*kernel_k2,
                                              X0 + scale_l*kernel_k2))
                bounds_ori = bounds_ori_temp.T
                bounds_ori_all.append(bounds_ori)

            bound_len = scale_l*kernel_k2
            temp = np.asarray(bounds_ori_all)
            temp_min = np.min(temp[:, :, 0], axis=0)
            temp_max = np.max(temp[:, :, 1], axis=0)
            bounds_ori = np.stack((temp_min, temp_max)).T
            bounds_new = bounds_ori.copy()
            print('Old bound: {}'.format(bounds))
            print('New bound: {}'.format(bounds_new))

            # Recompute bounds
            bounds = bounds_new.copy()
            bounds_iter.append(bounds)

            # Adjust iteration after adjusting bounds
            iter_adjust += (n_iter - 1)

            # Re-optimize the acquisition function with the new bound
            # and the new b_n
            n_iter = n_iter_i - iter_adjust 
            lengthscale = gp.kernel_.get_params()['k2__length_scale']
            thetan_2 = gp.kernel_.get_params()['k1__constant_value']
            radius = np.abs(np.max(bounds[:, 1]-bounds[:, 0]))
            b = 1/2*np.sqrt(2)*np.sqrt(thetan_2)/lengthscale
            a = 1
            tau_n = (4*myfunction.input_dim+4)*np.log(n_iter) + 2*np.log(2*np.pi**2/(3*sigma)) \
                    + 2*myfunction.input_dim*np.log(myfunction.input_dim*b*radius*np.sqrt(np.log(4*myfunction.input_dim*a/sigma)))
            b_n = np.sqrt(np.abs(nu*tau_n))

            x_max = acq_maximize_fixopt(gp, b_n, bounds)
            max_acq = acq_gp(x_max, gp, b_n)
            y_acq = acq_gp(np.asarray(X_init), gp, b_n)


            # Save some parameters of the bound
            X_init_bound = X_init.copy()
            Y_bound = Y.copy()
            lengthscale_bound = gp.kernel_.get_params()['k2__length_scale']
            scale_l_bound = scale_l

        # Check if the acquistion function argmax is at infinity
        # Then re-optimize within the bound that has the largest Y value
        if ((max_acq - b_n*np.sqrt(thetan_2)) >= -epsilon) & ((max_acq - b_n*np.sqrt(thetan_2)) <= 0):
            print('Re-optimize within smaller spheres')
            indices_max = np.argsort(Y_bound)[::-1]
            
            # Set a minimal number of local optimizations
            n_search = np.min([5, len(indices_max)])
            for i in range(n_search):
                X0 = X_init_bound[indices_max[i]]
                bounds_new = np.asarray((X0 - bound_len,
                                        X0 + bound_len))
                bounds_new = bounds_new.T
                x_max_local = acq_maximize_fixopt_local(gp, b_n, bounds_new, acq_type='ucb')
                max_acq_local = acq_gp(x_max_local, gp, b_n)
                max_inf = b_n*np.sqrt(gp.kernel_.get_params()['k1__constant_value'])
                if np.abs(max_acq_local-max_inf) <= epsilon:
                    x_max = x_max_local
                    break

        # Compute y_max
        y_max = func(x_max)
        Y_init = np.append(Y_init, y_max)
        X_init.append(x_max)
        Y = (Y_init-np.mean(Y_init))/(np.max(Y_init)-np.min(Y_init))

        # Update the kernel
        gp = GaussianProcessRegressor(kernel=kernel,
                                      n_restarts_optimizer=25)
        gp.fit(X_init, Y)
        gp_exp.append(gp)
        print('Kernel {} with Log Marginal Likelihood {}'.format(gp.kernel_, gp.log_marginal_likelihood(gp.kernel_.theta)))

        # Extract Gaussian Process hyper-parameters and update prior kernel
        # kernel_k2 is the length-scale
        # theta_n is the scale factor
        kernel_k1 = gp.kernel_.get_params()['k1__constant_value']
        kernel_k2 = gp.kernel_.get_params()['k2__length_scale']
        theta_n = np.sqrt(kernel_k1)
        kernel = kernel_k1*RBF(length_scale=kernel_k2)

        print('Maximum of y: {}'.format(np.max(Y_init)))
        Y_max[n_init_points+n_iter_i-1, 0] = np.max(Y_init)

    # End measuring time
    end = time.time()
    time_search_train = end-start
    print('Time cost (seconds): {:.2f}'.format(time_search_train))
    time_all.append(time_search_train)

    Y_max_all.append(Y_max)
    X_init_all.append(X_init)
    Y_init_all.append(Y_init)
    regret_all.append(regret_iter)
    r_optimal_all.append(r_optimal_iter)
    bounds_final_all.append(bounds_iter)

    '''
    # Save the list to files
    filename = 'results/result_' + bm_function + '_proposed_GPUBO_eps' +  str(epsilon).replace('.', '') + '.npz'
    np.savez(filename,
             *Y_max_all, *bounds_final_all, *X_init_all, *Y_init_all,
             *regret_all, *r_optimal_all, *bounds_final_all, *time_all)
    '''




## CLEAN-LABEL ATTACK

Problem analysis

Print a poisoned image

In [ ]:
print("Example of posioned sample")
plt.figure(figsize=(4,4))
plt.imshow(x_poisoned_train[indices_poisoned][44].reshape((32,32,3)))
plt.show()

print(x_poisoned_train[indices_poisoned][44][0][:10])

Print an adv image

In [ ]:
print("Example of posioned sample")
plt.figure(figsize=(4,4))
plt.imshow(final_advs[44].reshape((32,32,3)))
plt.show()

print(final_advs[44][0][:10])

Show how much of the original, adversarial and poisoned samples the model 2 is able to classify as the target label

In [ ]:
pred_adv = model2.predict(final_advs)
pred_adv_labels = [np.argmax(x) for x in pred_adv]

pred_poison = model2.predict(x_poisoned_train[indices_poisoned])
pred_poison_labels = [np.argmax(x) for x in pred_poison]

pred_original = model2.predict(X_train[indices_poisoned])
pred_original_labels = [np.argmax(x) for x in pred_original]

In [ ]:
print("Model 2 predict ", len([x for x in pred_adv_labels if x==target_class]),"/",final_advs.shape[0],"adversarial samples as label",target_class)

In [ ]:
print("Model 2 predict ", len([x for x in pred_poison_labels if x==target_class]),"/",final_advs.shape[0],"poisoned samples as label",target_class)

In [ ]:
print("Model 2 predict ", len([x for x in pred_original_labels if x==target_class]),"/",final_advs.shape[0],"original samples as label",target_class)

Function used to apply the trigger on an image without using ART

In [ ]:
def poison_dataset_clean_label_no_ART(x_clean, y_clean, x_poisoned, indices, x_pos, y_pos):
    '''
    Function to poison a dataset

    x_clean: vector of features of samples of the clean dataset
    y_clean: vector of labels of samples of the clean dataset
    x_pos: images among x_clean that have to be poisoned and added to x_clean
    y_pos: labels of x_pos
    indices: indices of x_pos among x_clean vector
    poison_func: inser_image function to insert the trigger in the image
    '''
    x_poison = np.copy(x_clean)
    is_poison = np.zeros(np.shape(y_clean)[0])

    poison_images=x_poisoned
    for i in range(indices.shape[0]):
        #set pixel "x_pos","y_pos" to red
        poison_images[i][x_pos][y_pos][0]=1
        poison_images[i][x_pos][y_pos][1]=0
        poison_images[i][x_pos][y_pos][2]=0
        
        #set pixel "x_pos+1","y_pos+1" to red
        poison_images[i][x_pos+1][y_pos+1][0]=1
        poison_images[i][x_pos+1][y_pos+1][1]=0
        poison_images[i][x_pos+1][y_pos+1][2]=0
        
        #set pixel "x_pos+2","y_pos+2" to red
        poison_images[i][x_pos+2][y_pos+2][0]=1
        poison_images[i][x_pos+2][y_pos+2][1]=0
        poison_images[i][x_pos+2][y_pos+2][2]=0
    
    x_poison[indices]=poison_images
    is_poison[indices]=1

    is_poison = is_poison != 0

    return is_poison, x_poison

Poison training dataset using the function defined in the cell before

In [ ]:
print("\n\nAttacking with position x:",x_pos," y:",y_pos,"and PP_POISON:",PP_POISON,"\n\n")
print("Poisoning training dataset...")

#poison dataset with backdoor trigger image
(is_poison_train,x_poisoned_train)=poison_dataset_clean_label_no_ART(X_train,y_train,final_advs,indices_poisoned,x_pos,y_pos)

print("Training dataset poisoned")